# Exercício 2



## Enunciado

Para a viga em balanço da figura abaixo tratada como um problema de Estado Plano de Tensões determine a matriz de rigidez e o vetor de forças para as seguintes discretizações:

a) Com 2 elementos triangulares lineares conforme mostrado nesta figura.

b) Com 1 elemento retangular com os nós 1,2,3 e 4.

Determine os deslocamentos livres considerando os nós 3 e 4 presos(deslocamentos nulos). 

Determine também os valores das tensões, segundo o sistema de eixos indicado nos pontos nodais e nos pontos centrais dos elementos considerados. Considere o carregamento indicado nesta figura com a distribuição de carga onde p = 150*(N+1)/20 . Adotar E (200.000) e Poisson(0.3) indicados na figura. Atente para trabalhar com unidades coerentes e que a espessura também está dada (t = 1). Utilize para as medidas do domínio o valor a = b = 4 + (N-1)/20


![](ex2.png)

## Resolução

### Substituindo N

Temos que $N = 1$, logo as dimensões do domínio serão:

$$
a = b = 4
$$

### Coordenadas dos nós

* Nó 1: $(4,4)$
* Nó 2: $(4,0)$
* Nó 3: $(0,4)$
* Nó 4: $(0,0)$

### a) Elementos Triangulares

#### Montagem da Matriz de Rigidez


A matriz de rigidez $K$ de um elemento triangular em estado plano de tensões é calculada como:

$$
K = tAB^T DB
$$

onde:

* $t$ é a espessura do elemento,
* $A$ é a área do elemento triangular,
* $B$ é a matriz de gradiente de deslocamento
* $D$ é a matriz de elasticidade plana.

A matriz $D$ é dada por:

$$
D = \dfrac{E}{1 - \nu^2}

\begin{bmatrix}
1 & \nu & 0 \\
\nu & 1 & 0 \\
0 & 0 & \dfrac{1 - \nu}{2}
\end{bmatrix}
$$

A matriz $B$ para um elemento triangular é derivada das coordenadas dos nós do elemento e é dada por:

$$
B = \dfrac{1}{2A} 

\begin{bmatrix}
b_1 & 0 & b_2 & 0 & b_3 & 0 \\
0 & c_1 & 0 & c_2 & 0 & c_3 \\
c_1 & b_1 & c_2 & b_2 & c_3 & b_3
\end{bmatrix}
$$



In [18]:
import numpy as np
import sympy as sp


# Propriedades do material
E = 200000  # Módulo de elasticidade (N/mm²)
nu = 0.3    # Coeficiente de Poisson
t = 1       # Espessura da viga (mm)

# Dimensões da viga
a = b = 4   # Dimensões para N = 1

# Carga aplicada
p = 150 * (1 + 1) / 20

# Coordenadas dos nós (em mm)
nodes = np.array([
    [a, b],  # Nó 1
    [a, 0],  # Nó 2
    [0, b],  # Nó 3
    [0, 0],  # Nó 4
])

# Elementos (triângulos) definidos pelos nós
elements = np.array([
    [1, 2, 3],  # Elemento 1
    [1, 3, 4]   # Elemento 2
])

# Número de nós e elementos
num_nodes = len(nodes)
num_elements = len(elements)

# Função para calcular a matriz de rigidez de um elemento triangular linear
def stiffness_matrix_triangle_linear(element, nodes, E, nu, t):
    # Coordenadas dos nós do elemento
    x = nodes[element - 1, 0]
    y = nodes[element - 1, 1]

    # Área do elemento
    A = 0.5 * np.linalg.det(np.array([[1, x[0], y[0]],
                                      [1, x[1], y[1]],
                                      [1, x[2], y[2]]]))

    # Matriz B
    B = 1 / (2 * A) * np.array([
        [y[1] - y[2], 0, y[2] - y[0], 0, y[0] - y[1], 0],
        [0, x[2] - x[1], 0, x[0] - x[2], 0, x[1] - x[0]],
        [x[2] - x[1], y[1] - y[2], x[0] - x[2], y[2] - y[0], x[1] - x[0], y[0] - y[1]]
    ])

    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    # Matriz de rigidez do elemento
    k = t * A * np.dot(np.dot(B.T, D), B)

    return k

# Inicialização da matriz de rigidez global
K_global = np.zeros((2 * num_nodes, 2 * num_nodes))

# Montagem da matriz de rigidez global
for element in elements:
    k_local = stiffness_matrix_triangle_linear(element, nodes, E, nu, t)
    for i in range(3):
        for j in range(3):
            for k in range(2):
                for l in range(2):
                    K_global[2 * (element[i] - 1) + k, 2 * (element[j] - 1) + l] += k_local[2 * i + k, 2 * j + l]

# Exibindo a matriz de rigidez global
display(sp.Matrix(K_global).evalf(3))

Matrix([
[-3.85e+4, -7.14e+4,  3.85e+4,  3.3e+4,        0,  7.14e+4,        0, -3.3e+4],
[-7.14e+4,  -1.1e+5,  3.85e+4,  1.1e+5,  7.14e+4,        0, -3.85e+4,       0],
[ 3.85e+4,  3.85e+4, -3.85e+4,       0,        0, -3.85e+4,        0,       0],
[  3.3e+4,   1.1e+5,        0, -1.1e+5,  -3.3e+4,        0,        0,       0],
[       0,  7.14e+4,        0, -3.3e+4,  3.85e+4, -7.14e+4, -3.85e+4,  3.3e+4],
[ 7.14e+4,        0, -3.85e+4,       0, -7.14e+4,   1.1e+5,  3.85e+4, -1.1e+5],
[       0, -3.85e+4,        0,       0, -3.85e+4,  3.85e+4,  3.85e+4,       0],
[ -3.3e+4,        0,        0,       0,   3.3e+4,  -1.1e+5,        0,  1.1e+5]])

#### Solução do Sistema de Equações


In [20]:
# Aplicação das condições de contorno (nós 3 e 4 fixos)
fixed_nodes = [3, 4]
free_dof = np.setdiff1d(np.arange(2 * num_nodes), np.array([2 * (node - 1) for node in fixed_nodes] + [2 * node - 1 for node in fixed_nodes]))

# Redução da matriz de rigidez e do vetor de forças para os graus de liberdade livres
K_reduced = K_global[np.ix_(free_dof, free_dof)]

# Vetor de forças externas (considerando a carga distribuída)
F = np.zeros(2 * num_nodes)
# Aplicando a carga no nó 2 (direção y)
F[2 * 2 - 1] = -p * a * b / 2  # Carga dividida igualmente entre os nós 2 e 3

# Redução do vetor de forças
F_reduced = F[free_dof]

# Resolvendo o sistema para obter os deslocamentos nos graus de liberdade livres
u_free = np.linalg.solve(K_reduced, F_reduced)

# Montagem do vetor de deslocamentos completo (incluindo zeros para graus de liberdade fixos)
u = np.zeros(2 * num_nodes)
u[free_dof] = u_free

print('Deslocamentos:')
for node in range(num_nodes):
    print('Nó {}: ({}, {})'.format(node + 1, u[2 * node], u[2 * node + 1]))


Deslocamentos:
Nó 1: (-0.0036399999999999887, -0.0031199999999999978)
Nó 2: (-0.006759999999999988, -0.003119999999999995)
Nó 3: (0.0, 0.0)
Nó 4: (0.0, 0.0)


#### Cálculo das Tensões

In [14]:
def stress_triangle_linear(element, nodes, u, E, nu):
    # Coordenadas dos nós do elemento
    x = nodes[element - 1, 0]
    y = nodes[element - 1, 1]

    # Área do elemento
    A = 0.5 * np.linalg.det(np.array([[1, x[0], y[0]],
                                      [1, x[1], y[1]],
                                      [1, x[2], y[2]]]))

    # Matriz B
    B = 1 / (2 * A) * np.array([
        [y[1] - y[2], 0, y[2] - y[0], 0, y[0] - y[1], 0],
        [0, x[2] - x[1], 0, x[0] - x[2], 0, x[1] - x[0]],
        [x[2] - x[1], y[1] - y[2], x[0] - x[2], y[2] - y[0], x[1] - x[0], y[0] - y[1]]
    ])

    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    # Vetor de deslocamentos do elemento
    u_element = np.array([u[2 * (node - 1):2 * node] for node in element]).flatten()

    # Cálculo das tensões
    stress = np.dot(D, np.dot(B, u_element))

    return stress

# Correção e cálculo das tensões
stresses = np.zeros((num_elements, 3))  # Inicializando o vetor de tensões

# Cálculo das tensões para cada elemento
for i, element in enumerate(elements):
    stresses[i] = stress_triangle_linear(element, nodes, u, E, nu)

print('Tensões nos elementos:')
for element in range(num_elements):
    print('Elemento {}: ({}, {}, {})'.format(element + 1, stresses[element, 0], stresses[element, 1], stresses[element, 2]))

Tensões nos elementos:
Elemento 1: (-5.592543275632488, 54.407456724367506, -5.592543275632486)
Elemento 2: (1.6777629826897453, 5.592543275632484, 5.592543275632495)


## b) Elemento Retangular

In [15]:
import numpy as np

# Propriedades do material
E = 200000  # Módulo de elasticidade (N/mm²)
nu = 0.3    # Coeficiente de Poisson
t = 1       # Espessura da viga (mm)

# Dimensões da viga para N = 1
a = b = 4   # Dimensões

# Carga aplicada
p = 150 * (1 + 1) / 20

# Coordenadas dos nós (em mm)
nodes = np.array([
    [0, 0],  # Nó 1
    [a, 0],  # Nó 2
    [a, b],  # Nó 3
    [0, b]   # Nó 4
])

# Número de nós
num_nodes = len(nodes)

# Função para calcular a matriz de rigidez de um elemento retangular
def stiffness_matrix_rectangular(E, nu, t, a, b):
    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    # Matriz de rigidez do elemento retangular
    k = t * a * b / 36 * np.array([
        [4, 2, -1, -2, -2, -1, -1, 0],
        [2, 4, 0, -1, -1, -2, -2, -1],
        [-1, 0, 4, 2, 1, 0, -2, -1],
        [-2, -1, 2, 4, 0, 1, -1, -2],
        [-2, -1, 1, 0, 4, 2, -1, 0],
        [-1, -2, 0, 1, 2, 4, 0, -1],
        [-1, -2, -2, -1, -1, 0, 4, 2],
        [0, -1, -1, -2, 0, -1, 2, 4]
    ])

    return k

# Aplicando a matriz de rigidez para um elemento retangular
K_rectangular = stiffness_matrix_rectangular(E, nu, t, a, b)

# Aplicando as condições de contorno (nós 3 e 4 fixos)
fixed_nodes = [3, 4]
free_dof_rect = np.setdiff1d(np.arange(2 * num_nodes), np.array([2 * (node - 1) for node in fixed_nodes] + [2 * node - 1 for node in fixed_nodes]))

# Redução da matriz de rigidez e do vetor de forças para os graus de liberdade livres
K_reduced_rect = K_rectangular[np.ix_(free_dof_rect, free_dof_rect)]

# Vetor de forças externas (considerando a carga distribuída)
F_rect = np.zeros(2 * num_nodes)
# Aplicando a carga no nó 2 (direção y)
F_rect[2 * 2 - 1] = -p * a * b / 2  # Carga dividida igualmente entre os nós 2 e 3

# Redução do vetor de forças
F_reduced_rect = F_rect[free_dof_rect]

# Resolvendo o sistema para obter os deslocamentos nos graus de liberdade livres
u_free_rect = np.linalg.solve(K_reduced_rect, F_reduced_rect)

# Montagem do vetor de deslocamentos completo (incluindo zeros para graus de liberdade fixos)
u_rect = np.zeros(2 * num_nodes)
u_rect[free_dof_rect] = u_free_rect

# Deslocamentos nos nós para o elemento retangular
display(sp.Matrix(u_rect.reshape((num_nodes, 2)).T).evalf(3))

Matrix([
[-41.1,   46.3, 0, 0],
[-7.71, -113.0, 0, 0]])

In [17]:
# Função para calcular as tensões em um elemento retangular
def stress_rectangular(E, nu, u, a, b):
    # Matriz de elasticidade plana
    D = E / (1 - nu**2) * np.array([
        [1, nu, 0],
        [nu, 1, 0],
        [0, 0, (1 - nu) / 2]
    ])

    # Gradientes de deslocamento
    Bx = 1 / (2 * a) * np.array([-1, 1, 1, -1])
    By = 1 / (2 * b) * np.array([-1, -1, 1, 1])

    # Deslocamentos nos nós do elemento
    u_x = u[::2]
    u_y = u[1::2]

    # Cálculo das deformações
    epsilon_x = np.dot(Bx, u_x)
    epsilon_y = np.dot(By, u_y)
    gamma_xy = np.dot(Bx, u_y) + np.dot(By, u_x)

    # Cálculo das tensões
    stress = np.dot(D, np.array([epsilon_x, epsilon_y, gamma_xy]))

    return stress

# Calculando as tensões no elemento retangular
stress_rect = stress_rectangular(E, nu, u_rect, a, b)
print('Tensões no elemento retangular: ({:.3f}, {}, {})'.format(stress_rect[0], stress_rect[1], stress_rect[2]))


Tensões no elemento retangular: (3397959.18367347, 4040816.3265306125, -1063186.8131868134)
